# Using Google Drive to Store Our Files

- Not using local storage anymore
- All files processed and uploaded to the cloud.



## Steps:

1) Authenticate with Google Drive
2) Try to upload an existing file to the cloud
3) Try to upload a file from the RSS feed to the cloud (idk if async work makes this possible)

In [ ]:
from dotenv import load_dotenv
# Google Authentication
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

gauth = GoogleAuth()
gauth.LocalWebserverAuth()
drive = GoogleDrive(gauth)

# Upload a file
fpath = 'Finished Episodes_Word/145. Bölüm - Burçin Acer.docx'
file = drive.CreateFile({'title': '145. Bölüm - Burçin Acer.docx'})
file.SetContentFile(fpath)
file.Upload()
print("File uploaded successfully")

## AssemblyAI -> Google Drive


In [ ]:
import os
import feedparser
import subprocess
import assemblyai as aai
import threading
from dotenv import load_dotenv


In [ ]:
nasil_olunur_url = "https://feeds.simplecast.com/TRRbx_5C"

load_dotenv()
ASSEMBLYAI_API_KEY = os.getenv("AAI_API_2")
aai.settings.api_key = ASSEMBLYAI_API_KEY
transcriptionID_dictionary = {}

In [ ]:
feed = feedparser.parse(nasil_olunur_url)
episodes = {}
for episode in feed.entries[16:78]: #5:78 is all
    title = episode["title"]
    audio_url = episode.enclosures[0].href
    episodes[title] = audio_url
    print(title, audio_url)
# Not: virgul veya - varsa birden fazla konuk oluyor


In [ ]:
config = aai.TranscriptionConfig(
    speaker_labels=True,
    speakers_expected=2, # bunu ileride otomatize et - simdilik 2+ speakerli textleri elle duzelt
    language_code="tr",
    entity_detection=True #entity detection ON
)

config.set_custom_spelling(
  {
    "Nilay": ["Nile"],
  }
)

In [ ]:
def transcribeAudio(audio_url):
    transcriber = aai.Transcriber(config=config)
    transcript = transcriber.transcribe(audio_url) #audiourl can be undefinded. check against it
    transcriptionID_dictionary[audio_url] = transcript.id

    #maybe I also need to write the transcript id to a file or something

In [ ]:
threads = []

for key,value in episodes.items():
    thread = threading.Thread(target=transcribeAudio, args=(value,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()

In [ ]:
word_episode_dict = {}

In [ ]:
for key, value in episodes.items():
    title = key
    guest = key.split("-")[1].strip()
    ep_no = key.split("-")[0].strip()
    transcript_id = transcriptionID_dictionary[value]
    transcript = aai.Transcript.get_by_id(transcript_id)
    word_episode_dict[title] = [title, guest, ep_no, transcript]

'''
title
guest
episode_no
transcript
'''

# Problem: transcript id ile bolumler eslenmiyor. Bunu nasil yapabilirim?

In [ ]:
# Write files as docx file
from docx import Document

new_transcription_folder = "/Users/kaanerdem/Desktop/projeler/pdtxt/pdtxt_notebook/transcripted_episodes/episodes_with_timestamps/new_episodes"

def write_to_word(transcript, title, guest, ep_no):
  file_path = os.path.join(new_transcription_folder, f"{title}.docx")

  doc = Document()
  doc.add_heading(f"Nasıl Olunur {ep_no}. Bölüm - {guest}", level=0)

  def time_conversion(millisec):
      second = (millisec // 1000) % 60
      minute = (millisec // 60000) % 60
      hour = millisec // 3600000

      return f"[{hour:02}:{minute:02}:{second:02}]"

  for utterance in transcript.utterances: # Once kim konusuyor kontrol edip ona gore speaker atamam lazim
    if utterance.speaker == "A":
      speaker = "Nilay Örnek"
    elif utterance.speaker == "B":
      speaker = guest

    time = time_conversion(utterance.start)

    paragraph = doc.add_paragraph()
    paragraph.add_run(f"{speaker} {time}: ").bold = True
    paragraph.add_run(utterance.text)


  doc.save(file_path)

In [ ]:
# Create Word Files
for key, value in word_episode_dict.items():
    write_to_word(value[3], value[0], value[1], value[2])

### Uploading to the AI Generated file Inside Nasil Olunur - Google Drive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os

GoogleDrive_FOLDER_ID = "GOOGLE_DRIVE_FOLDER_ID"

# Authenticate
gauth = GoogleAuth()
gauth.LocalWebserverAuth()  # Opens a web browser for authentication
drive = GoogleDrive(gauth)

# Folder containing files to upload
folder_path = "../transcripted_episodes/episodes_with_timestamps/new_episodes"

# Upload each file in the folder
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)

    if os.path.isfile(file_path):  # Ensure it's a file
        file = drive.CreateFile({'title': filename, 'parents': [{'id': GoogleDrive_FOLDER_ID}]})
        file.SetContentFile(file_path)
        file.Upload()
        print(f'Uploaded: {filename}')